In [1]:
import cv2
import mediapipe as mp
import numpy as np

In [2]:
import cv2

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)

    cv2.imshow("Camera Test", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [3]:
import cv2
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
ok, frame = cap.read()
print("ok:", ok)
cap.release()

ok: True


In [1]:
import cv2
import mediapipe as mp
import numpy as np
import collections
import time
from datetime import datetime

# açı hesaplama fonksiyonu
def calculate_angle(a, b, c):
    a, b, c = np.array(a), np.array(b), np.array(c)
    ba = a - b
    bc = c - b
    cos = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc) + 1e-6)
    angle = np.arccos(np.clip(cos, -1.0, 1.0))
    return np.degrees(angle)

def draw_card(frame, x, y, w, h, alpha=0.85, color=(15,15,25)):
    overlay = frame.copy()
    cv2.rectangle(overlay, (x, y), (x+w, y+h), color, -1)
    cv2.rectangle(overlay, (x, y), (x+w, y+h), (60,60,80), 2)
    frame = cv2.addWeighted(overlay, alpha, frame, 1 - alpha, 0)
    return frame

def put_text(frame, text, x, y, scale=0.6, color=(255,255,255), thickness=2, font=cv2.FONT_HERSHEY_DUPLEX):
    cv2.putText(frame, text, (x, y), font, scale, color, thickness, cv2.LINE_AA)


mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

knee_history = collections.deque(maxlen=10)
elbow_history = collections.deque(maxlen=10)
shoulder_elbow_distance_history = collections.deque(maxlen=15)
elbow_forward_history = collections.deque(maxlen=15)

#plank 
hip_height_history = collections.deque(maxlen=15)
body_angle_history = collections.deque(maxlen=15)

squat_reps = 0
curl_reps = 0
plank_hold_time = 0.0  

squat_state = "UP"
curl_state = "DOWN"
plank_state = "NOT_IN_POSITION" 

plank_start_time = None  # mükemmel pozisyona girince başla
plank_session_time = 0.0  # anlık durum süresi

exercise_mode = "unknown"

show_controls = True
paused = False

# kalori
total_calories = 0.0
CALORIES_PER_SQUAT = 0.32
CALORIES_PER_CURL = 0.18
CALORIES_PER_PLANK_SECOND = 0.05  # saniye başına kalori

# form skoru
form_quality_history = collections.deque(maxlen=30)
current_form_score = 100
squat_form_scores = []
curl_form_scores = []

current_alert = ""
alert_start_time = 0
alert_duration = 2.5
alert_cooldown = 1.0
last_alert_dict = {}

# fps hesaplama
prev_time = 0
fps_history = collections.deque(maxlen=10)

cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

print("Formcoach pro started")
print("Start exercising")
print("\nCONTROLS:")
print("  [Q] - Quit")
print("  [P] - Pause/Resume")
print("  [S] - Screenshot")
print("  [C] - Toggle Controls")
print("  [0] - Reset Counters")

with mp_pose.Pose(min_detection_confidence=0.5,
                  min_tracking_confidence=0.5) as pose:

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        frame = cv2.flip(frame, 1)
        h, w, c = frame.shape
        
        # fps hesaplama
        current_time = time.time()
        fps = 1 / (current_time - prev_time) if prev_time > 0 else 0
        prev_time = current_time
        fps_history.append(fps)
        fps_avg = int(np.mean(fps_history))

        knee_smooth = None
        elbow_smooth = None
        leg_visible = False
        arm_visible = False
        plank_visible = False
        extra_metrics = None
        confidence = 0
        wrist = None
        elbow_pos = None
        shoulder_pos = None

        if not paused:
            rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = pose.process(rgb)

            if results.pose_landmarks:
                lm = results.pose_landmarks.landmark
                mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

                visibilities = [landmark.visibility for landmark in lm]
                confidence = int(np.mean(visibilities) * 100)

                # squat
                hip_lm = lm[mp_pose.PoseLandmark.RIGHT_HIP]
                knee_lm = lm[mp_pose.PoseLandmark.RIGHT_KNEE]
                ankle_lm = lm[mp_pose.PoseLandmark.RIGHT_ANKLE]

                if hip_lm.visibility > .7 and knee_lm.visibility > .7 and ankle_lm.visibility > .7:
                    leg_visible = True
                    hip = (hip_lm.x*w, hip_lm.y*h)
                    knee = (knee_lm.x*w, knee_lm.y*h)
                    ankle = (ankle_lm.x*w, ankle_lm.y*h)

                    knee_angle = calculate_angle(hip, knee, ankle)
                    knee_history.append(knee_angle)
                    knee_smooth = int(np.mean(knee_history))

                    shoulder_lm = lm[mp_pose.PoseLandmark.RIGHT_SHOULDER]
                    foot_lm = lm[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX]

                    shoulder_pos_squat = (shoulder_lm.x*w, shoulder_lm.y*h)
                    foot = (foot_lm.x*w, foot_lm.y*h)

                    trunk_angle = calculate_angle((hip[0], hip[1]-150), hip, shoulder_pos_squat)
                    shin_angle = calculate_angle((ankle[0], ankle[1]-150), ankle, knee)
                    trunk_tibia = trunk_angle - shin_angle

                    knee_toe_dx = knee[0] - foot[0]
                    tibia_len = np.hypot(knee[0]-ankle[0], knee[1]-ankle[1])
                    knee_toe_ratio = knee_toe_dx / (tibia_len + 1e-6)

                    extra_metrics = {
                        "trunk_angle": trunk_angle,
                        "shin_angle": shin_angle,
                        "trunk_tibia_diff": trunk_tibia,
                        "knee_toe_ratio": knee_toe_ratio,
                    }

                # curl
                shoulder_lm = lm[mp_pose.PoseLandmark.RIGHT_SHOULDER]
                elbow_lm = lm[mp_pose.PoseLandmark.RIGHT_ELBOW]
                wrist_lm = lm[mp_pose.PoseLandmark.RIGHT_WRIST]
                hip_lm = lm[mp_pose.PoseLandmark.RIGHT_HIP]

                if shoulder_lm.visibility > .7 and elbow_lm.visibility > .7 and wrist_lm.visibility > .7:
                    arm_visible = True

                    shoulder_pos = (shoulder_lm.x*w, shoulder_lm.y*h)
                    elbow_pos = (elbow_lm.x*w, elbow_lm.y*h)
                    wrist = (wrist_lm.x*w, wrist_lm.y*h)
                    hip = (hip_lm.x*w, hip_lm.y*h)

                    elbow_angle = calculate_angle(shoulder_pos, elbow_pos, wrist)
                    elbow_history.append(elbow_angle)
                    elbow_smooth = int(np.mean(elbow_history))

                    shoulder_elbow_dist = np.hypot(
                        shoulder_pos[0] - elbow_pos[0],
                        shoulder_pos[1] - elbow_pos[1]
                    )
                    shoulder_elbow_distance_history.append(shoulder_elbow_dist)

                    elbow_forward = elbow_pos[0] - shoulder_pos[0]
                    elbow_forward_history.append(elbow_forward)

                # plank
                left_shoulder_lm = lm[mp_pose.PoseLandmark.LEFT_SHOULDER]
                right_shoulder_lm = lm[mp_pose.PoseLandmark.RIGHT_SHOULDER]
                left_hip_lm = lm[mp_pose.PoseLandmark.LEFT_HIP]
                right_hip_lm = lm[mp_pose.PoseLandmark.RIGHT_HIP]
                left_ankle_lm = lm[mp_pose.PoseLandmark.LEFT_ANKLE]
                right_ankle_lm = lm[mp_pose.PoseLandmark.RIGHT_ANKLE]
                left_elbow_lm = lm[mp_pose.PoseLandmark.LEFT_ELBOW]
                right_elbow_lm = lm[mp_pose.PoseLandmark.RIGHT_ELBOW]

                # plank  kontrolü 
                plank_landmarks_visible = (
                    left_shoulder_lm.visibility > .4 and right_shoulder_lm.visibility > .4 and
                    left_hip_lm.visibility > .4 and right_hip_lm.visibility > .4 and
                    left_ankle_lm.visibility > .4 and right_ankle_lm.visibility > .4
                )

                if plank_landmarks_visible:
                    # koordinatörler
                    l_shoulder = (left_shoulder_lm.x*w, left_shoulder_lm.y*h)
                    r_shoulder = (right_shoulder_lm.x*w, right_shoulder_lm.y*h)
                    l_hip = (left_hip_lm.x*w, left_hip_lm.y*h)
                    r_hip = (right_hip_lm.x*w, right_hip_lm.y*h)
                    l_ankle = (left_ankle_lm.x*w, left_ankle_lm.y*h)
                    r_ankle = (right_ankle_lm.x*w, right_ankle_lm.y*h)
                    l_elbow = (left_elbow_lm.x*w, left_elbow_lm.y*h)
                    r_elbow = (right_elbow_lm.x*w, right_elbow_lm.y*h)

                    # ortalama noktalar
                    avg_shoulder = ((l_shoulder[0]+r_shoulder[0])/2, (l_shoulder[1]+r_shoulder[1])/2)
                    avg_hip = ((l_hip[0]+r_hip[0])/2, (l_hip[1]+r_hip[1])/2)
                    avg_ankle = ((l_ankle[0]+r_ankle[0])/2, (l_ankle[1]+r_ankle[1])/2)
                    avg_elbow = ((l_elbow[0]+r_elbow[0])/2, (l_elbow[1]+r_elbow[1])/2)

                    # omuz-kalça-ayak açısı
                    body_line_angle = calculate_angle(avg_shoulder, avg_hip, avg_ankle)
                    body_angle_history.append(body_line_angle)

                    # kalça yüksekliğine bakıyoruz 
                    # y ekseninde omuz kalça ayak orta noktası
                    reference_y = (avg_shoulder[1] + avg_ankle[1]) / 2
                    hip_height_deviation = abs(avg_hip[1] - reference_y)
                    hip_height_history.append(hip_height_deviation)

                    # kollar bükülü mü diye dirsek açısına bakıyoruz
                    left_elbow_angle = calculate_angle(l_shoulder, l_elbow, (l_elbow[0], l_elbow[1]+100))
                    right_elbow_angle = calculate_angle(r_shoulder, r_elbow, (r_elbow[0], r_elbow[1]+100))
                    
                    # y ekseninin farkına bakıcaz
                    #yataysa küçük, dikeyse büyük
                    vertical_span = abs(avg_shoulder[1] - avg_ankle[1])
                    horizontal_span = abs(avg_shoulder[0] - avg_ankle[0])
                    
                    is_horizontal = horizontal_span > vertical_span * 0.5
                    
                    # dirsekler omuzdan aşağıda mı diye bakıyoruz ki düz durmak plank algınlanmasın
                    elbows_below_shoulders = avg_elbow[1] > avg_shoulder[1] - 30  #  tolerans ekledim hata aldım çalıştırınca

                    #plank düz
                    in_plank_position = (
                        150 < body_line_angle < 210 and    
                        is_horizontal and  # vücut yatay olacak
                        elbows_below_shoulders # dirsekler aşağıda
                    )

                    if in_plank_position:
                        plank_visible = True

            # plank öncelikli

            knee_range = max(knee_history) - min(knee_history) if len(knee_history) > 4 else 0
            elbow_range = max(elbow_history) - min(elbow_history) if len(elbow_history) > 4 else 0

            # hareket yoksa plank olabilir
            if plank_visible:
                exercise_mode = "plank"
            elif leg_visible and knee_range > 25 and not plank_visible:
                exercise_mode = "squat"
            elif arm_visible and elbow_range > 25 and not plank_visible:
                exercise_mode = "curl"

            if exercise_mode == "squat" and knee_smooth:
                if squat_state == "UP" and knee_smooth < 110:
                    squat_state = "DOWN"
                if squat_state == "DOWN" and knee_smooth > 150:
                    squat_state = "UP"
                    squat_reps += 1
                    total_calories += CALORIES_PER_SQUAT
                    if len(form_quality_history) > 0:
                        rep_score = int(np.mean(form_quality_history))
                        squat_form_scores.append(rep_score)

            if exercise_mode == "curl" and elbow_smooth:
                if curl_state == "DOWN" and elbow_smooth < 60:
                    curl_state = "UP"
                if curl_state == "UP" and elbow_smooth > 150:
                    curl_state = "DOWN"
                    curl_reps += 1
                    total_calories += CALORIES_PER_CURL
                    if len(form_quality_history) > 0:
                        rep_score = int(np.mean(form_quality_history))
                        curl_form_scores.append(rep_score)

            # plank logic
            if exercise_mode == "plank" and plank_visible:
                # form kontrolü
                body_angle_smooth = np.mean(body_angle_history) if len(body_angle_history) > 5 else 180
                hip_height_smooth = np.mean(hip_height_history) if len(hip_height_history) > 5 else 0


                is_perfect_form = (
                    160 < body_angle_smooth < 200 and  # toleranslı
                    hip_height_smooth < 120  # yüksek sapma toleransı
                )

                if is_perfect_form:
                    plank_state = "PERFECT"
                    if plank_start_time is None:
                        plank_start_time = time.time()
                    else:
                        # saniye hesaplama
                        elapsed = time.time() - plank_start_time
                        plank_session_time = elapsed
                        plank_hold_time += 1/fps_avg if fps_avg > 0 else 0.033
                        total_calories += CALORIES_PER_PLANK_SECOND / fps_avg if fps_avg > 0 else CALORIES_PER_PLANK_SECOND * 0.033
                else:
                    plank_state = "BAD_FORM"
                    if plank_start_time is not None:
                        elapsed_since_start = time.time() - plank_start_time
                        if elapsed_since_start < 1.0:
                            plank_state = "PERFECT"
                        else:
                            plank_start_time = None
                            plank_session_time = 0.0
            else:
                if exercise_mode != "plank" and plank_start_time is not None:
                    plank_start_time = None
                    plank_session_time = 0.0
                plank_state = "NOT_IN_POSITION"

        panel_w = 380
        panel_h = 75
        frame = draw_card(frame, 10, 10, panel_w, panel_h, 0.88, (12,15,22))
        
        status_text = "PAUSED" if paused else "FormCoach"
        status_color = (255,100,100) if paused else (255,255,255)
        put_text(frame, status_text, 25, 35, 0.65, status_color, 2)
        put_text(frame, f"Camera: {fps_avg} FPS", 25, 60, 0.48, (180,180,180), 1)
        put_text(frame, f"Detection: {confidence}%", 220, 60, 0.48, (180,180,180), 1)
        
        panel2_y = 95
        panel2_h = 155
        frame = draw_card(frame, 10, panel2_y, panel_w, panel2_h, 0.88, (12,15,22))
        
        if exercise_mode == "squat":
            put_text(frame, "MODE: SQUAT", 25, panel2_y + 32, 0.8, (100,200,255), 2)
        elif exercise_mode == "curl":
            put_text(frame, "MODE: BICEPS CURL", 25, panel2_y + 32, 0.7, (255,120,180), 2)
        elif exercise_mode == "plank":
            put_text(frame, "MODE: PLANK", 25, panel2_y + 32, 0.8, (255,200,100), 2)
        else:
            put_text(frame, "MODE: WAITING...", 25, panel2_y + 32, 0.7, (150,150,150), 2)
        
        put_text(frame, f"Squat: {squat_reps}", 25, panel2_y + 65, 0.65, (255,255,255), 2)
        
        put_text(frame, f"Biceps: {curl_reps}", 220, panel2_y + 65, 0.65, (255,255,255), 2)
        
        put_text(frame, f"Plank: {plank_hold_time:.1f}s", 25, panel2_y + 95, 0.65, (255,255,255), 2)
        
        put_text(frame, f"Calories: {total_calories:.1f} kcal", 25, panel2_y + 120, 0.6, (100,255,150), 2)
        
        if current_form_score >= 85:
            score_color = (100, 255, 100)
        elif current_form_score >= 70:
            score_color = (100, 200, 255)
        elif current_form_score >= 50:
            score_color = (100, 255, 255)
        else:
            score_color = (100, 150, 255)
        
        put_text(frame, f"Form Quality: {current_form_score}/100", 25, panel2_y + 145, 0.6, score_color, 2)

        if exercise_mode == "plank" and plank_state == "PERFECT":
            timer_w = 350
            timer_x = (w - timer_w) // 2
            frame = draw_card(frame, timer_x, 15, timer_w, 75, 0.92, (10,30,10))
            
            cv2.rectangle(frame, (timer_x, 15), (timer_x+timer_w, 90), (100,255,100), 3)
            
            put_text(frame, "PERFECT PLANK!", timer_x + 60, 45, 0.7, (100,255,100), 2)
            put_text(frame, f"Time: {plank_session_time:.1f}s", timer_x + 90, 75, 0.8, (255,255,255), 2)

        if not paused:
            new_alert = ""
            form_score = 100

            # sadece squat modundayken
            if exercise_mode == "squat" and extra_metrics:
                if extra_metrics["trunk_angle"] > 45:
                    new_alert = "Torso too forward!"
                    form_score -= 25
                elif extra_metrics["knee_toe_ratio"] > 0.40:
                    new_alert = "Knees over toes!"
                    form_score -= 30
                elif extra_metrics["trunk_tibia_diff"] < -10:
                    new_alert = "Knee-dominant squat!"
                    form_score -= 20
                
                if extra_metrics["trunk_angle"] > 35:
                    form_score -= 10
                if extra_metrics["knee_toe_ratio"] > 0.30:
                    form_score -= 10

            if exercise_mode == "curl" and elbow_smooth and arm_visible and wrist and elbow_pos and shoulder_pos:
                if len(elbow_forward_history) >= 10:
                    avg_forward = np.mean(list(elbow_forward_history)[-10:])
                    if avg_forward > 60:
                        new_alert = "Elbow too far forward!"
                        form_score -= 25
                    elif avg_forward > 45:
                        form_score -= 15
                
                if len(shoulder_elbow_distance_history) >= 10:
                    dist_values = list(shoulder_elbow_distance_history)
                    current_dist = dist_values[-1]
                    baseline_dist = np.mean(dist_values[:5]) if len(dist_values) >= 5 else current_dist
                    
                    if current_dist > baseline_dist * 1.2:
                        new_alert = "Keep shoulder stable!"
                        form_score -= 20
                    elif current_dist > baseline_dist * 1.15:
                        form_score -= 10
                
                if curl_state == "DOWN" and elbow_smooth and elbow_smooth < 135:
                    new_alert = "Extend arm fully!"
                    form_score -= 15

            if exercise_mode == "plank" and plank_visible:
                body_angle_smooth = np.mean(body_angle_history) if len(body_angle_history) > 5 else 180
                hip_height_smooth = np.mean(hip_height_history) if len(hip_height_history) > 5 else 0

                if body_angle_smooth < 160:
                    new_alert = "Hips too high! Lower them"
                    form_score -= 30
                elif body_angle_smooth > 200:
                    new_alert = "Hips sagging! Raise them"
                    form_score -= 30
                elif hip_height_smooth > 150:
                    new_alert = "Keep body in straight line!"
                    form_score -= 20

            form_quality_history.append(max(0, form_score))
            current_form_score = int(np.mean(form_quality_history)) if len(form_quality_history) > 0 else 100

            current_time_alert = time.time()
            
            if new_alert:
                can_show = True
                if new_alert in last_alert_dict:
                    time_since_last = current_time_alert - last_alert_dict[new_alert]
                    if time_since_last < alert_cooldown:
                        can_show = False
                
                if can_show:
                    current_alert = new_alert
                    alert_start_time = current_time_alert
                    last_alert_dict[new_alert] = current_time_alert
            
            if current_alert:
                elapsed = current_time_alert - alert_start_time
                if elapsed > alert_duration:
                    current_alert = ""

        # perfect plank değilse
        if current_alert and not (exercise_mode == "plank" and plank_state == "PERFECT"):
            alert_w = 450
            alert_x = (w - alert_w) // 2
            alert_y = 15 if exercise_mode != "plank" else h - 100
            frame = draw_card(frame, alert_x, alert_y, alert_w, 60, 0.88, (25,15,5))
            put_text(frame, f"{current_alert}", alert_x + 25, alert_y + 35, 0.75, (255,170,0), 2)

        # kontrol paneli
        if show_controls:
            ctrl_w = 270
            ctrl_h = 160
            ctrl_x = w - ctrl_w - 15
            ctrl_y = h - ctrl_h - 15
            
            frame = draw_card(frame, ctrl_x, ctrl_y, ctrl_w, ctrl_h, 0.88, (12,15,22))
            
            put_text(frame, "CONTROLS", ctrl_x + 15, ctrl_y + 28, 0.55, (255,255,255), 2)
            cv2.line(frame, (ctrl_x+15, ctrl_y+35), (ctrl_x+ctrl_w-15, ctrl_y+35), (60,60,70), 1)
            
            controls = [
                ("[Q]", "Quit"),
                ("[P]", "Pause/Resume"),
                ("[S]", "Screenshot"),
                ("[C]", "Hide Controls"),
                ("[0]", "Reset Counter")
            ]
            
            y_offset = ctrl_y + 58
            for key, desc in controls:
                put_text(frame, key, ctrl_x + 20, y_offset, 0.48, (100,200,255), 1)
                put_text(frame, desc, ctrl_x + 75, y_offset, 0.48, (200,200,200), 1)
                y_offset += 22

        cv2.namedWindow("FormCoach Pro", cv2.WINDOW_NORMAL)
        cv2.resizeWindow("FormCoach Pro", 1280, 720)
        cv2.imshow("FormCoach Pro", frame)
        
        key = cv2.waitKey(1) & 0xFF
        
        if key == ord("q") or key == ord("Q"):
            break
            
        elif key == ord("s") or key == ord("S"):
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"formcoach_{timestamp}.png"
            cv2.imwrite(filename, frame)
            print(f"📸 Screenshot saved: {filename}")
            
        elif key == ord("c") or key == ord("C"):
            show_controls = not show_controls
            status = "hidden" if not show_controls else "visible"
            print(f"Control panel {status}")
            
        elif key == ord("0"):
            squat_reps = 0
            curl_reps = 0
            plank_hold_time = 0.0
            plank_session_time = 0.0
            total_calories = 0.0
            squat_form_scores = []
            curl_form_scores = []
            form_quality_history.clear()
            plank_start_time = None
            print(" Counters reset")
            
        elif key == ord("p") or key == ord("P"):
            paused = not paused
            status = "paused" if paused else "resumed"
            print(f"⏸️ Program {status}")

cap.release()
cv2.destroyAllWindows()

Formcoach pro started
Start exercising

CONTROLS:
  [Q] - Quit
  [P] - Pause/Resume
  [S] - Screenshot
  [C] - Toggle Controls
  [0] - Reset Counters


C:\Users\dilar\AppData\Local\Programs\Python\Python310\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
